In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan,when,count,mean,sum,split,isnull,avg

spark = SparkSession.builder.appName("Q3").getOrCreate()

df = spark.read.csv("Cleaned_DS_Jobs.csv",header=True,inferSchema=True)


df = df.withColumn("min_salary", split(col("Salary Estimate"), "-").getItem(0).cast("int"))
df = df.withColumn("max_salary", split(col("Salary Estimate"), "-").getItem(1).cast("int"))

df = df.withColumn("Avg Salary",((col("min_salary")+col("max_salary"))/2).cast("double"))


df = df.withColumn(
    "Rating",
    when(col("Rating") == -1, 1)
    .when(col("Rating") == 0, 1)
    .otherwise(col("Rating")),
)

"""
null_check = df.select(
    [
        count(when(col(c).isNull() | isnan(c) | (col(c) == ""), c)).alias(c)
        for c in df.columns
    ]   
)

null_check.show()
"""

for column in df.columns:
    df = df.withColumn(column, when(isnull(col(column)), -1).otherwise(col(column)))


# Q4

average_salary_per_job = df.groupBy("Job Title").agg(avg("Avg Salary").alias("Avg Salary per job"))

average_salary_per_job.show()


# Q5

df = df.filter((col("Size")!="-1") & (col("Size")!="Unknown"))
"""

distinct_elements = df.select("Size").distinct().collect()


for _ in distinct_elements:
    print(_['Size'])

    
"""

size_category = when(
    col("Size").rlike('1 to 50|51 to 200'), 'Small'
).when(
    col("Size").rlike('201 to 500|501 to 1000|1001 to 5000'), 'Medium'
).when(
    col("Size").rlike('5001 to 10000|10000\+'), 'Large'
).otherwise('Unknown')


df = df.withColumn("SizeCategory", size_category)

avg_salary_on_category = df.groupBy("SizeCategory").agg(avg("avg salary").alias('avg_salary_on_category'))

avg_salary_on_category.show()

# average_salary_per_size = df.groupBy("Size").agg(avg("Avg Salary").alias("Avg Salary per Size"))


# average_salary_per_job.show()


spark.stop()

+--------------------+------------------+
|           Job Title|Avg Salary per job|
+--------------------+------------------+
|Senior Data Scien...| 99.33333333333333|
|Clinical Data Ana...|             164.5|
|Senior Business I...|              90.0|
|Data Analyst/Engi...|             115.5|
|Staff BI and Data...|             107.0|
|Intelligence Data...|             90.75|
|Report Writer-Dat...|              92.5|
|Hydrogen/Tritium ...|             148.0|
|Business Intellig...|            109.25|
|        Data Modeler|             154.0|
|Scientist / Group...|             197.5|
|Senior Research S...|             105.0|
|Software Engineer...|             164.5|
|   Sr Data Scientist|            126.75|
|COMPUTER SCIENTIS...|             271.5|
|Data Scientist/Ma...|             125.5|
|Data Scientist - ...|             120.5|
|  Decision Scientist|              94.5|
|Data Scientist - ...|             97.75|
|Data Scientist / ...|             128.5|
+--------------------+------------